In [ ]:
import cv2
import torch
import time
from typing import Optional

def load_yolo_model(model_name: str = 'yolov5x') -> torch.nn.Module:
    """
    Load the YOLOv5 model from Ultralytics' PyTorch Hub.
    
    Args:
        model_name (str): The model variant to load. Default is 'yolov5x'.
    
    Returns:
        torch.nn.Module: The YOLOv5 model.
    """
    return torch.hub.load('ultralytics/yolov5', model_name)

def process_video(video_path: str, output_path: str, resize_factor: float = 0.5) -> None:
    """
    Process a video to detect cars using YOLOv5 and save the output with annotations.
    
    Args:
        video_path (str): Path to the input video file.
        output_path (str): Path to save the output video file.
        resize_factor (float): Factor to resize the video. Default is 0.5.
    """
    # Load YOLOv5 model
    model = load_yolo_model()
    
    # Initialize video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    # Get original dimensions
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Calculate new dimensions
    new_width = int(original_width * resize_factor)
    new_height = int(original_height * resize_factor)
    
    # Get the original frame rate of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))
    
    # Class index for cars in COCO dataset (YOLOv5 uses this)
    car_class_index = 2
    
    # Calculate the time between frames
    frame_time = 1.0 / fps
    
    while cap.isOpened():
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize the frame
        frame_resized = cv2.resize(frame, (new_width, new_height))
        
        # Perform inference
        results = model(frame_resized)
        
        # Extract detections
        detections = results.xyxy[0].cpu().numpy()
        annotated_frame = frame_resized.copy()
        
        # Filter for cars only
        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            if int(cls) == car_class_index:
                cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(annotated_frame, f'Car {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Display the frame
        cv2.imshow('Car Detection', annotated_frame)
        
        # Write the frame to output video
        out.write(annotated_frame)
        
        # Calculate elapsed time
        elapsed_time = time.time() - start_time
        
        # Wait to synchronize with the video frame rate
        if elapsed_time < frame_time:
            time.sleep(frame_time - elapsed_time)
        
        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    input_video_path = '4K Road traffic video for object detection and tracking - free download now! [MNn9qKG2UFI].webm'
    output_video_path = 'car_detection_objects.avi'
    process_video(input_video_path, output_video_path)
